In [ ]:
import numpy as np
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
data_path = "/content/drive/MyDrive/Data/"

In [ ]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

In [ ]:
#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data)
y_test_tensor = torch.LongTensor(test_label)

In [ ]:
# Load your training data
x_train = np.load(data_path + 'train_data.npy')
y_train = np.load(data_path + 'train_label.npy')


In [ ]:
# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.Tensor(x_train)
y_train_tensor = torch.LongTensor(y_train)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [ ]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

In [ ]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)
        x = self.classifier(x)
        return x

# Define a list of classes corresponding to the predicted indices
classes = ["hello", "help me", "stop", "thank you", "yes"]

num_classes = 5
model = EEGAutoencoderClassifier(num_classes).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 523
for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

In [ ]:
model.eval()
correct = 0
total = 0
predicted_words = []

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Convert predicted indices to words
        predicted_words.extend([classes[i] for i in predicted])

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print("Predicted words:", predicted_words)

Test Accuracy: 52.13%
Predicted words: ['help me', 'stop', 'help me', 'stop', 'yes', 'yes', 'help me', 'stop', 'stop', 'hello', 'stop', 'yes', 'yes', 'help me', 'help me', 'stop', 'help me', 'yes', 'stop', 'stop', 'stop', 'stop', 'help me', 'stop', 'stop', 'hello', 'hello', 'stop', 'stop', 'help me', 'stop', 'thank you', 'yes', 'thank you', 'yes', 'help me', 'stop', 'yes', 'yes', 'stop', 'stop', 'thank you', 'yes', 'stop', 'help me', 'stop', 'stop', 'help me', 'stop', 'yes', 'stop', 'thank you', 'stop', 'thank you', 'help me', 'stop', 'yes', 'help me', 'thank you', 'stop', 'thank you', 'stop', 'yes', 'thank you', 'yes', 'yes', 'yes', 'hello', 'yes', 'hello', 'hello', 'hello', 'thank you', 'yes', 'hello', 'stop', 'yes', 'stop', 'yes', 'thank you', 'stop', 'yes', 'yes', 'stop', 'stop', 'hello', 'help me', 'stop', 'stop', 'stop', 'stop', 'yes', 'stop', 'thank you', 'stop', 'help me', 'thank you', 'thank you', 'stop', 'stop', 'hello', 'thank you', 'yes', 'yes', 'thank you', 'thank you', 't

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open('/content/drive/MyDrive/SavedModel', 'wb'))   #saving the trained model

In [ ]:
loaded = pickle.load(open('/content/drive/MyDrive/SavedModel', 'rb'))  #openig the traied model

In [ ]:
loaded.eval()         # evaluating loaded model accuracy
correct = 0
total = 0
predicted_words = []

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Convert predicted indices to words
        predicted_words.extend([classes[i] for i in predicted])

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print("Predicted words:", predicted_words)

Test Accuracy: 52.13%
Predicted words: ['help me', 'stop', 'help me', 'stop', 'yes', 'yes', 'help me', 'stop', 'stop', 'hello', 'stop', 'yes', 'yes', 'help me', 'help me', 'stop', 'help me', 'yes', 'stop', 'stop', 'stop', 'stop', 'help me', 'stop', 'stop', 'hello', 'hello', 'stop', 'stop', 'help me', 'stop', 'thank you', 'yes', 'thank you', 'yes', 'help me', 'stop', 'yes', 'yes', 'stop', 'stop', 'thank you', 'yes', 'stop', 'help me', 'stop', 'stop', 'help me', 'stop', 'yes', 'stop', 'thank you', 'stop', 'thank you', 'help me', 'stop', 'yes', 'help me', 'thank you', 'stop', 'thank you', 'stop', 'yes', 'thank you', 'yes', 'yes', 'yes', 'hello', 'yes', 'hello', 'hello', 'hello', 'thank you', 'yes', 'hello', 'stop', 'yes', 'stop', 'yes', 'thank you', 'stop', 'yes', 'yes', 'stop', 'stop', 'hello', 'help me', 'stop', 'stop', 'stop', 'stop', 'yes', 'stop', 'thank you', 'stop', 'help me', 'thank you', 'thank you', 'stop', 'stop', 'hello', 'thank you', 'yes', 'yes', 'thank you', 'thank you', 't